# Chest Disease Detection - Multi-Label Classification

This notebook implements a multi-label classification pipeline for detecting 9 different chest X-ray conditions:

1. **Atelectasis** - การปอดแฟบ
2. **Cardiomegaly** - โรคหัวใจโต
3. **Consolidation** - การแข็งตัวของปอด
4. **Edema** - อาการบวมน้ำในปอด
5. **Enlarged Cardio mediastinum** - การขยายตัวของหัวใจและเยื่อหุ้มหัวใจ
6. **Fracture** - กระดูกหัก
7. **Lung Lesion** - รอยโรคในปอด
8. **Lung Opacity** - ความขุ่นข้นของปอด
9. **No Finding** - ไม่พบความผิดปกติ

The model uses ConvNeXTv2 architecture for multi-label classification with sigmoid activation.

In [3]:
!pip -q install torch torchvision transformers timm

In [22]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from timm import create_model

In [33]:
class ChestDiseaseDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        
        # Define the 9 disease classes
        self.disease_columns = [
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
            'Enlarged Cardio mediastinum', 'Fracture', 'Lung Lesion',
            'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
            'Pneumonia','Pneumothorax'
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get image path from first column
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        
        # Get multi-label targets (13 classes)
        labels = torch.tensor(
            self.data.iloc[idx, 1:14].values.astype(float), 
            dtype=torch.float32
        )

        if self.transform:
            image = self.transform(image)

        return image, labels

In [34]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Flip images randomly
    transforms.RandomRotation(10),  # Rotate images slightly
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [35]:
image_dir = "/kaggle/input/individual-test-chest-disease-detection/images/images"
train_dir = "/kaggle/input/individual-test-chest-disease-detection/train.csv"

train_dataset = ChestDiseaseDataset(train_dir, image_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Number of batches: {len(train_loader)}")

# Check data shape
sample_image, sample_labels = train_dataset[0]
print(f"Image shape: {sample_image.shape}")
print(f"Labels shape: {sample_labels.shape}")
print(f"Sample labels: {sample_labels}")

Training dataset size: 9963
Number of batches: 623
Image shape: torch.Size([3, 224, 224])
Labels shape: torch.Size([13])
Sample labels: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])


In [36]:
import timm

# timm.list_models("")

In [37]:
# Create model for multi-label classification (9 classes)
model = create_model("convnextv2_base", pretrained=True, num_classes=13)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model created with {sum(p.numel() for p in model.parameters())} parameters")
print(f"Using device: {device}")

# Print model architecture summary
print("\nModel output shape for batch size 1:")
dummy_input = torch.randn(1, 3, 224, 224).to(device)
with torch.no_grad():
    output = model(dummy_input)
    print(f"Output shape: {output.shape}")  # Should be [1, 9]

model.safetensors:   0%|          | 0.00/355M [00:00<?, ?B/s]

Model created with 87706125 parameters
Using device: cuda

Model output shape for batch size 1:
Output shape: torch.Size([1, 13])


In [38]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

print("Loss function: BCEWithLogitsLoss (suitable for multi-label)")
print("Optimizer: Adam with weight decay")
print("Scheduler: ReduceLROnPlateau")

Loss function: BCEWithLogitsLoss (suitable for multi-label)
Optimizer: Adam with weight decay
Scheduler: ReduceLROnPlateau


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [39]:
def train_model(model, dataloader, criterion, optimizer, scheduler, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    
    # Disease class names for monitoring
    disease_names = [
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
            'Enlarged Cardio mediastinum', 'Fracture', 'Lung Lesion',
            'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
            'Pneumonia','Pneumothorax'
        ]

    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = torch.zeros(13)  # Track accuracy per class
        total_predictions = 0
        
        for batch_idx, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Calculate accuracy for each class
            predictions = torch.sigmoid(outputs) > 0.5
            correct_predictions += (predictions == labels.bool()).sum(dim=0).cpu()
            total_predictions += labels.size(0)
            
            # Print progress every 50 batches
            if batch_idx % 50 == 0:
                print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        # Calculate epoch metrics
        epoch_loss = running_loss / len(dataloader)
        class_accuracies = correct_predictions / total_predictions
        
        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Average Loss: {epoch_loss:.4f}")
        print("Class Accuracies:")
        for i, (name, acc) in enumerate(zip(disease_names, class_accuracies)):
            print(f"  {name}: {acc:.4f}")
        
        # Update learning rate
        scheduler.step(epoch_loss)
        print(f"Current LR: {optimizer.param_groups[0]['lr']:.6f}")
        print("-" * 50)

In [47]:
train_model(model, train_loader, criterion, optimizer, scheduler, epochs=5)

Epoch 1, Batch 0, Loss: 0.1836
Epoch 1, Batch 50, Loss: 0.2267
Epoch 1, Batch 100, Loss: 0.2210
Epoch 1, Batch 150, Loss: 0.2795
Epoch 1, Batch 200, Loss: 0.3086
Epoch 1, Batch 250, Loss: 0.3076
Epoch 1, Batch 300, Loss: 0.2689
Epoch 1, Batch 350, Loss: 0.2218
Epoch 1, Batch 400, Loss: 0.2402
Epoch 1, Batch 450, Loss: 0.2804
Epoch 1, Batch 500, Loss: 0.1978
Epoch 1, Batch 550, Loss: 0.2615
Epoch 1, Batch 600, Loss: 0.2032

Epoch 1/5
Average Loss: 0.2425
Class Accuracies:
  Atelectasis: 0.8561
  Cardiomegaly: 0.8791
  Consolidation: 0.9539
  Edema: 0.9363
  Enlarged Cardio mediastinum: 0.9495
  Fracture: 0.9503
  Lung Lesion: 0.9118
  Lung Opacity: 0.8138
  No Finding: 0.8255
  Pleural Effusion: 0.9088
  Pleural Other: 0.9661
  Pneumonia: 0.8843
  Pneumothorax: 0.9402
Current LR: 0.000100
--------------------------------------------------
Epoch 2, Batch 0, Loss: 0.2453
Epoch 2, Batch 50, Loss: 0.2348
Epoch 2, Batch 100, Loss: 0.2347
Epoch 2, Batch 150, Loss: 0.3304
Epoch 2, Batch 200, L

In [48]:
torch.save(model.state_dict(), "convnextv2_10epochs_chest_disease_model.pth")
print("Model saved as 'convnextv2_10epochs_chest_disease_model.pth'")

Model saved as 'convnextv2_10epochs_chest_disease_model.pth'


# Multi-Label Prediction Pipeline

This section implements prediction for the 9 chest disease classes. The model outputs probabilities for each class, and we can set different thresholds for each class or use a global threshold of 0.5.

In [49]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [50]:
import torch
import os
import pandas as pd
from PIL import Image
from torchvision import transforms

def predict(model, test_folder, output_csv, threshold=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Disease class names
    disease_columns = [
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
            'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
            'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
            'Pneumonia','Pneumothorax'
        ]
    
    test_images = pd.read_csv(output_csv)["filename"]
    results = []

    with torch.no_grad():
        for img_name in test_images:
            img_path = os.path.join(test_folder, img_name)
            image = Image.open(img_path).convert('RGB')
            image = test_transform(image).unsqueeze(0).to(device)
            
            # Get model output and apply sigmoid
            outputs = model(image)
            probabilities = torch.sigmoid(outputs).cpu().numpy()[0]
            
            # Convert probabilities to binary predictions using threshold
            predictions = (probabilities > threshold).astype(int)
            
            # Create result row: image_id + 9 class predictions
            result_row = [os.path.splitext(img_name)[0] + ".jpg"] + predictions.tolist()
            results.append(result_row)
            
            # Print sample prediction for debugging
            if len(results) <= 3:
                print(f"Image: {img_name}")
                print("Probabilities:")
                for disease, prob in zip(disease_columns, probabilities):
                    print(f"  {disease}: {prob:.4f}")
                print("Predictions:", predictions)
                print("-" * 30)

    # Create DataFrame with proper column names
    columns = ['id'] + disease_columns
    df = pd.DataFrame(results, columns=columns)
    df = df.rename(columns={"id": "filename"})
    df.to_csv("10_epochs_finalist.csv", index=False)
    
    print(f"\nPredictions saved to {output_csv}")
    print(f"Total images processed: {len(test_images)}")
    print(f"Columns in output: {list(df.columns)}")
    
    # Show prediction statistics
    print("\nPrediction Statistics:")
    for disease in disease_columns:
        positive_count = df[disease].sum()
        print(f"  {disease}: {positive_count} positive predictions ({positive_count/len(df)*100:.1f}%)")
    
    return df

In [51]:
prediction_df = predict(model, image_dir, "/kaggle/input/individual-test-chest-disease-detection/test_submission.csv", threshold=0.5)

Image: cxr00001.jpg
Probabilities:
  Atelectasis: 0.7170
  Cardiomegaly: 0.1562
  Consolidation: 0.0142
  Edema: 0.0651
  Enlarged Cardiomediastinum: 0.0461
  Fracture: 0.0478
  Lung Lesion: 0.0022
  Lung Opacity: 0.3375
  No Finding: 0.0094
  Pleural Effusion: 0.9341
  Pleural Other: 0.0023
  Pneumonia: 0.0806
  Pneumothorax: 0.0833
Predictions: [1 0 0 0 0 0 0 0 0 1 0 0 0]
------------------------------
Image: cxr00002.jpg
Probabilities:
  Atelectasis: 0.1002
  Cardiomegaly: 0.0194
  Consolidation: 0.0172
  Edema: 0.0103
  Enlarged Cardiomediastinum: 0.1385
  Fracture: 0.0271
  Lung Lesion: 0.0339
  Lung Opacity: 0.1955
  No Finding: 0.0079
  Pleural Effusion: 0.8609
  Pleural Other: 0.1171
  Pneumonia: 0.0206
  Pneumothorax: 0.2329
Predictions: [0 0 0 0 0 0 0 0 0 1 0 0 0]
------------------------------
Image: cxr00003.jpg
Probabilities:
  Atelectasis: 0.0194
  Cardiomegaly: 0.3766
  Consolidation: 0.0049
  Edema: 0.0107
  Enlarged Cardiomediastinum: 0.0201
  Fracture: 0.4956
  Lung L

In [52]:
final_list = pd.read_csv("10_epochs_finalist.csv")
len(final_list)

2506

In [45]:
# Additional utility functions for analysis

def analyze_predictions(df):
    """Analyze the prediction results"""
    disease_columns = [
        'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
        'Enlarged Cardio mediastinum', 'Fracture', 'Lung Lesion',
        'Lung Opacity', 'No Finding'
    ]
    
    print("=== Prediction Analysis ===")
    print(f"Total images: {len(df)}")
    print(f"Average diseases per image: {df[disease_columns].sum(axis=1).mean():.2f}")
    
    # Check for images with no findings
    no_disease_images = df[df[disease_columns].sum(axis=1) == 0]
    print(f"Images with no predicted diseases: {len(no_disease_images)}")
    
    # Check for images with multiple diseases
    multiple_disease_images = df[df[disease_columns].sum(axis=1) > 1]
    print(f"Images with multiple diseases: {len(multiple_disease_images)}")
    
    return df

def predict_single_image(model, image_path, disease_columns):
    """Predict diseases for a single image"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    image = Image.open(image_path).convert('RGB')
    image = test_transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.sigmoid(outputs).cpu().numpy()[0]
    
    print(f"Predictions for {image_path}:")
    for disease, prob in zip(disease_columns, probabilities):
        status = "✓" if prob > 0.5 else "✗"
        print(f"  {status} {disease}: {prob:.4f}")
    
    return probabilities

# Example usage (uncomment to test on a single image):
# disease_columns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
#                   'Enlarged Cardio mediastinum', 'Fracture', 'Lung Lesion',
#                   'Lung Opacity', 'No Finding']
# predict_single_image(model, "test/sample_image.jpg", disease_columns)